# Génération de texte avec réseaux de neurones récurrents LSTM avec Keras

#### Par : BOUTAHIR Mohamed Khalifa [ MASTER IPS ]
##### Data : nous allons utiliser un livre de l'enfance comme Data: "Alice's Adventures in Wonderland de Lewis Carroll" . https://gist.github.com/phillipj/4944029
##### Le but de projet est de predire une suite où un nouveau scenario pour ce livre .L'utilisation des réseaux LSTM est pour apprendre des séquences de personnages d’Alice au pays des merveilles. Et par la suite le modèle va générer des nouvelles séquences de caractères.
<img src="wonderland.jpg" width=400 />

## 1- Importation des bibliothèques et le jeu de données

In [1]:
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


# 2- Préparation de l’ensemble de données

#### Importation du fichier texte et le convertir en minuscule

In [2]:
filename = "wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

#### Detecter les caractères de jeu de donnees et les classer 
 Nous ne pouvons pas modéliser les caractères directement dans le réseau de neurones, nous devons plutôt convertir les caractères en entiers.

In [3]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

#### Calcule le nombre total des caractères  de texte et le nombre des caractères 

In [4]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  148574
Total Vocab:  46


# 2- Creation de réseau de neurones

Nous allons diviser le texte du livre en sous-séquences d'une longueur fixe de 100 caractères. <br>
Chaque modèle d'apprentissage du réseau est composé de 100 pas de temps d'un caractère (X) suivis d'une sortie de caractère (y).

In [2]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

NameError: name 'n_chars' is not defined

In [6]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [7]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [8]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [9]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
148474/148474 [==============================] - 2088s 14ms/step - loss: 2.9417

Epoch 00001: loss improved from inf to 2.94174, saving model to weights-improvement-01-2.9417.hdf5
Epoch 2/20
148474/148474 [==============================] - 2085s 14ms/step - loss: 2.7057

Epoch 00002: loss improved from 2.94174 to 2.70566, saving model to weights-improvement-02-2.7057.hdf5
Epoch 3/20
148474/148474 [==============================] - 1873s 13ms/step - loss: 2.6061

Epoch 00003: loss improved from 2.70566 to 2.60609, saving model to weights-improvement-03-2.6061.hdf5
Epoch 4/20
148474/148474 [==============================] - 2203s 15ms/step - loss: 2.5378

Epoch 00004: loss improved from 2.60609 to 2.53778, saving model to weights-improvement-04-2.5378.hdf5
Epoch 5/20
148474/148474 [==============================] - 1971s 13ms/step - loss: 2.4771

Epoch 00005: loss improved from 2.53778 to 2.47714, saving model to weights-improvement-05-2.4771.hdf5
Epoch 6/20
146816/148474 [===

In [39]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(500):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print ("\nDone.")

Seed:
" 

  just then her head struck against the roof of the hall:  in
fact she was now more than nine feet "
 hare    `and the ture oi the toies oi the sooel ' 
  `thet did toe tore the sime!' she manch hare taid to alice, and the tart on thine the was oo the was wottenn to the thet was ano ariel and then so her hend and the tabde so sal at the could, and she taid to she care pateng toiee an the woode and all the tooe of the tooe of the tooe of the tooe of the tooe of the tooe of the tooe of the tooe of the tooe of the tooe of the tooe of the tooe of the tooe of the tooe of the tooe of the tooe of the 
Done.
